# Change these flags to train a specific model

In [ ]:
TRAIN_RESNET = False
TRAIN_UNODE = True
TRAIN_UNET = False

---

In [ ]:
import os
import glob
import random

import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import torch
import torch.utils.data
import torch.nn.functional as F

from PIL import Image
from tqdm import tqdm
from torch import nn

%matplotlib inline

from models import ConvODEUNet, ConvResUNet, ODEBlock, Unet

import torchvision
import scipy.ndimage
import torch.optim as optim

from tqdm import tqdm, tqdm_notebook
from scipy.misc import imresize
from IPython.display import clear_output
from skimage.measure import label

# Commands to download the dataset

In [ ]:
!wget https://warwick.ac.uk/fac/sci/dcs/research/tia/glascontest/download/warwick_qu_dataset_released_2016_07_08.zip

In [ ]:
!unzip warwick_qu_dataset_released_2016_07_08.zip

## Define datasets

In [ ]:
cv2.setNumThreads(0)

In [ ]:
from augmentations import ElasticTransformations, RandomRotationWithMask

In [ ]:
class GLaSDataLoader(object):
    def __init__(self, patch_size, dataset_repeat=1, images=np.arange(0, 70), validation=False):
        self.image_fname = 'Warwick QU Dataset (Released 2016_07_08)/train_' 
        self.images = images
        
        self.patch_size = patch_size
        self.repeat = dataset_repeat
        self.validation = validation
        
        self.image_mask_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomVerticalFlip(),
            RandomRotationWithMask(20, resample=False, expand=False, center=None),
            ElasticTransformations(2000, 60),
            torchvision.transforms.ToTensor()
        ])
        self.image_transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.1, hue=0.1),
            torchvision.transforms.ToTensor()
        ])

    def __getitem__(self, index):
        # generate filename using index
        index_img = index // self.repeat
        index_img = self.images[index_img]
        index_str = str(index_img + 1)
        
        image = self.image_fname + index_str + '.bmp'
        mask = self.image_fname + index_str + '_anno.bmp'
        
        # open and resize image and mask
        image = Image.open(image)
        ratio = (775 / 512)
        new_size = (int(round(image.size[0] / ratio)), 
                    int(round(image.size[1] / ratio)))
        
        image = image.resize(new_size)
        
        mask = Image.open(mask)
        mask = mask.resize(new_size)
        
        image = np.array(image)
        mask = np.array(mask)
        
        # pad images if needed
        if not self.validation:
            pad_h = max(self.patch_size[0] - image.shape[0], 128) 
            pad_w = max(self.patch_size[1] - image.shape[1], 128)
        else: 
            # we pad more during training for data augmentation by translation 
            pad_h = max((self.patch_size[0] - image.shape[0]) // 2 + 1, 0)
            pad_w = max((self.patch_size[1] - image.shape[1]) // 2 + 1, 0)
            
        # pad to image size
        padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='reflect')
        mask = np.pad(mask, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')

        # randomly pick location when training
        if not self.validation:
            loc_y = random.randint(0, padded_image.shape[0] - self.patch_size[0])        
            loc_x = random.randint(0, padded_image.shape[1] - self.patch_size[1])  
        else:
            loc_y = 0
            loc_x = 0
            
        new_mask = np.zeros(mask.shape)

        # create borders
        for i in np.unique(mask):
            if i == 0: continue
            gland_mask = (mask == i).astype(np.float32)
            binarized_mask_border = gland_mask - ndimage.morphology.binary_erosion(gland_mask, iterations=5, border_value=1)
            gland_mask = gland_mask * 2 - binarized_mask_border
            new_mask[mask == i] = gland_mask[mask == i]
            
        mask = new_mask
        patch = torch.from_numpy(padded_image.transpose(2, 0, 1)).float() / 255
        label = torch.from_numpy(mask) / 2
                
        # perform data augmentation on image and mask
        if not self.validation:            
            patch_label_concat = torch.cat((patch, label[None, :, :].float()))
            patch_label_concat = self.image_mask_transforms(patch_label_concat)
            patch, label = patch_label_concat[0:3], np.round(patch_label_concat[3] * 2)
            patch = self.image_transforms(patch)
        else:
            label *= 2
            
        # crop to patch size
        patch = patch[:, 
                      loc_y:loc_y+self.patch_size[0],
                      loc_x:loc_x+self.patch_size[1]]   
        label = label[loc_y:loc_y+self.patch_size[0],
                      loc_x:loc_x+self.patch_size[1]]
        
        
        return patch, label.long()

    def __len__(self):
        return len(self.images) * self.repeat

Windows users: you may need to put the dataloader inside a different python file if using multiprocessing.

In [ ]:
trainset = GLaSDataLoader((352, 512), dataset_repeat=1)
valset = GLaSDataLoader((352, 512), dataset_repeat=1, images=np.arange(80, 85), validation=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=6)
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=6)

# Plotting train data

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=6, figsize=(24, 15))

for y in range(5):
    for x in range(3):
        sample = trainset[y]
        ax[y, x * 2].imshow(sample[0].numpy().transpose(1,2,0))
        ax[y, x * 2 + 1].imshow(sample[1])
        ax[y, x * 2].axis('off')
        ax[y, x * 2 + 1].axis('off')

plt.show(); 

# Plotting validation data

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=6, figsize=(24, 15))

for y in range(5):
    for x in range(3):
        sample = valset[y]
        ax[y, x * 2].imshow(sample[0].numpy().transpose(1,2,0))
        ax[y, x * 2 + 1].imshow(sample[1])
        ax[y, x * 2].axis('off')
        ax[y, x * 2 + 1].axis('off')

plt.show(); 

# Define network

In [ ]:
device = torch.device('cuda')

if TRAIN_UNODE:
    hidden_dim = 16
    net = ConvODEUNet(device, (3, 512, 512), hidden_dim, time_dependent=True, 
                      non_linearity='lrelu', augment_dim=hidden_dim - 3, adjoint=True, tol=1e-3)
    net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
if TRAIN_RESNET:
    net = ConvResUNet(device, (3, 512, 512), 16, non_linearity='lrelu', output_dim=3)
    net.to(device)

In [ ]:
if TRAIN_UNET:
    net = Unet(5, 3, 64, 3)
    net.to(device)

---

In [ ]:
for m in net.modules():
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.bias, 0)

In [ ]:
net.__class__

In [ ]:
net

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(net)

# Train model

In [ ]:
criterion = nn.CrossEntropyLoss()

if TRAIN_UNET:
    cross_entropy = torch.nn.CrossEntropyLoss()
    def criterion(conf, labels):
        out_shape = conf.shape[2:4]
        label_shape = labels.shape[1:3]

        w = (label_shape[1] - out_shape[1]) // 2 
        h = (label_shape[1] - out_shape[1]) // 2
        dh, dw = out_shape[0:2]
        conf_loss = cross_entropy(conf, labels[:, h:h+dh, w:w+dw].long())

        return conf_loss

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
losses = []
val_losses = []
nfe = [[],[],[],[],[],[],[],[],[]]

In [ ]:
from tqdm import tqdm

In [ ]:
accumulate_batch = 8  # mini-batch size by gradient accumulation
accumulated = 0

if TRAIN_RESNET: filename = 'best_border_resnet_model.pt'
elif TRAIN_UNODE: filename = 'best_border_unode_model.pt'
elif TRAIN_UNET: filename = 'best_border_unet_model.pt'

def run(lr=1e-3, epochs=100):
    accumulated = 0
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for epoch in range(epochs):
        
        # training loop with gradient accumulation
        running_loss = 0.0
        optimizer.zero_grad()
        for data in tqdm(trainloader):
            inputs, labels = data[0].cuda(), data[1].cuda()
            outputs = net(inputs)
            loss = criterion(outputs, labels) / accumulate_batch
            loss.backward()
            accumulated += 1
            if accumulated == accumulate_batch:
                optimizer.step()
                optimizer.zero_grad()
                accumulated = 0

            running_loss += loss.item() * accumulate_batch

        losses.append(running_loss / len(trainloader))
        
        # validation loop
        with torch.no_grad():
            running_loss = 0.0
            for data in valloader:
                inputs, labels = data[0].cuda(), data[1].cuda()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item()

            val_losses.append(running_loss / len(valloader))
            if np.argmin(val_losses) == len(val_losses) - 1 and loss < 0.4:
                torch.save(net, filename)
                
            plot_losses()
                
def plot_losses():
    # plot statistics
    if TRAIN_UNODE:
        nfe[0].append(net.odeblock_down1.odefunc.nfe)
        nfe[1].append(net.odeblock_down2.odefunc.nfe)
        nfe[2].append(net.odeblock_down3.odefunc.nfe)
        nfe[3].append(net.odeblock_down4.odefunc.nfe)
        nfe[4].append(net.odeblock_embedding.odefunc.nfe)
        nfe[5].append(net.odeblock_up1.odefunc.nfe)
        nfe[6].append(net.odeblock_up2.odefunc.nfe)
        nfe[7].append(net.odeblock_up3.odefunc.nfe)
        nfe[8].append(net.odeblock_up4.odefunc.nfe)

    clear_output(wait=True)

    if TRAIN_UNODE: cols = 4
    else: cols = 3
    fig, ax = plt.subplots(nrows=1, ncols=cols, figsize=(15,5))

    if TRAIN_UNODE: fig.suptitle('U-NODE', fontsize=16)
    elif TRAIN_RESNET: fig.suptitle('RESNET', fontsize=16)
    elif TRAIN_UNET: fig.suptitle('UNET', fontsize=16)

    ax[0].plot(np.arange(len(losses)), losses, label="loss")
    ax[0].plot(np.arange(len(val_losses)), val_losses, label="val_loss")

    if TRAIN_UNODE:
        ax[3].plot(np.arange(len(nfe[0])), nfe[0], label="down1")
        ax[3].plot(np.arange(len(nfe[0])), nfe[1], label="down2")
        ax[3].plot(np.arange(len(nfe[0])), nfe[2], label="down3")
        ax[3].plot(np.arange(len(nfe[0])), nfe[3], label="down4")
        ax[3].plot(np.arange(len(nfe[0])), nfe[4], label="embed")
        ax[3].plot(np.arange(len(nfe[0])), nfe[5], label="up1")
        ax[3].plot(np.arange(len(nfe[0])), nfe[6], label="up2")
        ax[3].plot(np.arange(len(nfe[0])), nfe[7], label="up3")
        ax[3].plot(np.arange(len(nfe[0])), nfe[8], label="up4")
        ax[3].legend() 


    outputs = torch.argmax(torch.softmax(outputs, dim=1), dim=1)[0]
    outputs = outputs.detach().cpu()
    outputs = outputs.numpy()

    ax[0].legend() 
    ax[1].imshow(outputs)
    ax[2].imshow(inputs.detach().cpu()[0].numpy().transpose(1,2,0))

    plt.show();

In [ ]:
if TRAIN_UNODE or TRAIN_RESNET: lr = 1e-4 
else: lr = 1e-5

run(lr, 1200 - len(losses)) # 300 epochs

# Calculate results

In [ ]:
# load best model
net = torch.load(filename)

In [ ]:
with torch.no_grad():
    running_loss = 0.0
    for data in tqdm(valloader):
        inputs, labels = data[0].cuda(), data[1].cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

    print("Check validation loss:", running_loss / len(valloader))

# Visualize results on validation set

In [ ]:
from inference_utils import resize_image, pad_image, prepare_image, crop_result, inference_image, eval_image

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=3, figsize=(4*3,3*5))

ax[0, 0].set_title('Image')
ax[0, 1].set_title('Ground-truth')
ax[0, 2].set_title('Trained network')

for col in range(3):
    for row in range(5):
        image = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/train_{row+81}.bmp')
        ann = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/train_{row+81}_anno.bmp')
        
        with torch.no_grad():
            segmentation, input_image = inference_image(image)
        if col == 0:
            ax[row, col].imshow(input_image)
        elif col == 1:
            ax[row, col].imshow(np.array(ann) > 0)
        else:
            ax[row, col].imshow(input_image)
            ax[row, col].imshow(segmentation, alpha=0.5)
                
        ax[row, col].set_axis_off()

plt.show(); 

# Calculate metrics on test set

In [ ]:
from metrics import ObjectDice, ObjectHausdorff, F1score

In [ ]:
TEST_RESNET = False
TEST_UNODE = True
TEST_UNET = False

In [ ]:
if TEST_UNODE: net = torch.load('best_border_unode_model.pt')
elif TEST_RESNET: net = torch.load('best_border_resnet_model.pt')
elif TEST_UNET: net = torch.load('best_border_unet_model.pt')

In [ ]:
testA_images = np.arange(60) + 1
testB_images = np.arange(20) + 1

if TEST_UNODE: folder = 'results_unode'
elif TEST_UNET: folder = 'results_unet'
elif TEST_RESNET: folder = 'results_resnet'
    
os.makedirs(folder)

for index in tqdm(testA_images):
    image = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/testA_{index}.bmp')
    result, resized = inference_image(image, TEST_UNET)
    pil_img = Image.fromarray(result.astype(np.uint8) * 255)
    pil_img.save(f'./{folder}/{index}.png')
    
for index in tqdm(testB_images):
    image = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/testB_{index}.bmp')
    result, resized = inference_image(image, TEST_UNET)
    pil_img = Image.fromarray(result.astype(np.uint8) * 255)
    pil_img.save(f'./{folder}/{index+60}.png')

In [ ]:
dice = 0
hausdorff = 0
f1 = 0

testA_images = np.arange(60) + 1
testB_images = np.arange(20) + 1

if TRAIN_UNODE: folder = 'results_unode'
elif TRAIN_UNET: folder = 'results_unet'
elif TRAIN_RESNET: folder = 'results_resnet'

for i in tqdm_notebook(testA_images):
    gt = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/testA_{i}_anno.bmp')
    result = Image.open(f'{folder}/{i}.png')

    result = result.resize(gt.size)
    labeled_result = label(np.array(result))
    
    Image.fromarray(labeled_result.astype(np.uint8)).save(f'./{folder}/{i}_labeled.png')

    f1 += F1score(labeled_result, gt)
    hausdorff += ObjectHausdorff(labeled_result, gt)
    dice += ObjectDice(labeled_result, gt)

diceA = dice 
hausdorffA = hausdorff 
f1A = f1

for i in tqdm_notebook(testB_images):
    gt = Image.open(f'augmented-neural-odes-master/Warwick QU Dataset (Released 2016_07_08)/testB_{i}_anno.bmp')
    result = Image.open(f'{folder}/{i + 60}.png')

    result = result.resize(gt.size)
    labeled_result = label(np.array(result))
    
    Image.fromarray(labeled_result.astype(np.uint8)).save(f'./{folder}/{i+60}_labeled.png')
    
    f1 += F1score(labeled_result, gt)
    hausdorff += ObjectHausdorff(labeled_result, gt)
    dice += ObjectDice(labeled_result, gt)

diceB = dice - diceA
hausdorffB = hausdorff - hausdorffA
f1B = f1 - f1A

print('ObjectDice:', dice / 80, 'A', diceA / 60, 'B', diceB / 20)
print('Hausdorff:', hausdorff / 80, 'A', hausdorffA / 60, 'B', hausdorffB / 20)
print('F1:', f1 / 80, 'A', f1A / 60, 'B', f1B / 20)